# prologue

### set up notebook and load package

In [1]:
# load what we need
import numpy as np
import CHIRPS.datasets as ds
import CHIRPS.reproducible as rp
import multiprocessing as mp

## Sensitivity Analysis

In [ ]:
# Optional Memory and Computation Cost Management
# CHIRPS is time economical but memory intensive to compute for lots of instances at once.
forest_walk_async=True
chirps_explanation_async=True
n_cores = mp.cpu_count()-2

# How many instances from the test set do you want to explain?
# A number bigger than the test set will be interpreted as 'all'
n_instances = 100
start_instance = 0 # here can opt to start at a specific instance, diagnostic if something crashes

model = 'RandomForest'
# model = 'AdaBoost1'
# model = 'AdaBoost2'
# model = 'GBM'

# prepare data
datasets = [
#         ds.car,
#         ds.cardio,
#         ds.credit,
#         ds.german,
#         ds.lending_tiny_samp,
#         ds.nursery,
#         ds.bankmark,
#         ds.rcdv,
        ds.adult,
       ]

# location to save results
project_dir = '/datadisk/whiteboxing/benchmarks2'
# project_dir = 'V:\\whiteboxing\\tests' # defaults to a directory "whiteboxing" in the working directory
# project_dir = 'C:\\Users\\Crutt\\Documents\\whiteboxing\\tests'

random_state_splits = 123 # change this if you want to try different splits of the data into test / train
random_state_rf = 123 # change this if you want to try with different forest construction
random_state_exp = 123 # change this if you want to try with different runs of the explainer algorithm (affects bootstrap eval)

verbose = True

# CHIRPS default parameters - see papers for details
precis_threshold = 0.99
merging_bootstraps = 20 # how many training bootstraps to test improvement in growing rule?
pruning_bootstraps = 20 # how many training bootstraps to test deterioration in pruning rule?
delta = 0.09 # pruning deterioration tolerance paramater

tuning = {'override' : False}

if model == 'RandomForest':
    tuning.update({'grid' : {'n_estimators': [(i + 1) * 200 for i in range(8)], 'max_depth' : [32]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=random_state_rf,
                                           random_state_splits=random_state_splits,
                                           start_instance=start_instance,
                                           verbose=verbose, n_cores=n_cores)

    alpha_paths = np.tile([0.9, 0.5, 0.1], 24)
    disc_path_bins = np.tile(np.repeat([4, 8], 3), 12)
    score_func = np.tile(np.repeat([5, 3, 1], 6), 4)
    support_paths = np.tile(np.repeat([0.3, 0.2], 18), 2)
    weighting = np.repeat(['kldiv', 'nothing'], 36)

    kwargs_grid = {k : {'alpha_paths' : ap, 'disc_path_bins' : dpb, 'disc_path_eqcounts' : False,
                        'score_func' : sf, 'weighting' : w, 'support_paths' : sp,
                        'precis_threshold' : precis_threshold,
                        'merging_bootstraps' : merging_bootstraps,
                        'pruning_bootstraps' : pruning_bootstraps,
                        'which_trees' : 'majority',
                        'delta' : delta} 
        for k, ap, dpb, sf, w, sp in zip(range(72), alpha_paths, disc_path_bins, score_func, weighting, support_paths)}
    
    
    for kwargs in kwargs_grid:
        bi_copy = rp.deepcopy(benchmark_items) # to avoid running down the internal counters
        control = {'method' : 'CHIRPS', 'model' : model,
                    'n_instances' : n_instances,
                    'random_state' : random_state_exp,
                    'kwargs' : kwargs_grid[kwargs],
                    'forest_walk_async' : forest_walk_async,
                    'chirps_explanation_async' : chirps_explanation_async,
                    'n_cores' : n_cores,
                    'save_sensitivity_path' : 'rf_sensitivity'}
        
        print('Sensitivity round: ' + str(kwargs))
        rp.do_benchmarking(bi_copy, verbose, **control)
        
elif model in ('AdaBoost1', 'AdaBoost2'):
    if model == 'AdaBoost1':
        algo = 'SAMME'
        save_sensitivity_path = 'ada1_sensitivity'
        support_paths = np.tile(np.tile([0.05, 0.02, 0.01], 16), 2)
        n_kwargs = 36
    else:
        algo = 'SAMME.R'
        save_sensitivity_path = 'ada2_sensitivity'
        support_paths = np.tile(np.tile([0.005, 0.002, 0.001], 16), 2)
        n_kwargs = 72
        
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=123, random_state_splits=123,
                                           start_instance=start_instance,
                                           verbose=verbose, n_cores=n_cores)
    
    disc_path_bins = np.tile(np.tile(np.repeat([4, 8], 3), 6), 2)
    disc_path_eqcounts = np.tile(np.tile(np.repeat([True, False], 6), 6), 2)
    weighting = np.tile(np.repeat(['chisq', 'kldiv', 'nothing'], 12), 2)
    which_trees = np.repeat(['majority', 'conf_weighted'], 36)

    kwargs_grid = {k : {'paths_lengths_threshold' : 5, 'alpha_paths' : 0.0,
                        'disc_path_bins' : dpb, 'disc_path_eqcounts' : dpeq,
                        'score_func' : 1, 'weighting' : w, 'support_paths' : sp,
                        'merging_bootstraps' : merging_bootstraps,
                        'pruning_bootstraps' : pruning_bootstraps,
                        'which_trees' : wchtr,
                        'delta' : delta} 
    for k, dpb, dpeq, w, sp, wchtr \
                   in zip(range(n_kwargs), disc_path_bins, disc_path_eqcounts, weighting, support_paths, which_trees)}

    for kwargs in kwargs_grid: # range(24, 36): #
        bi_copy = rp.deepcopy(benchmark_items) # to avoid running down the internal counters
        control = {'method' : 'CHIRPS', 'model' : model,
                    'n_instances' : n_instances,
                    'random_state' : random_state_exp,
                    'kwargs' : kwargs_grid[kwargs],
                    'forest_walk_async' : forest_walk_async,
                    'chirps_explanation_async' : chirps_explanation_async,
                    'n_cores' : n_cores,
                    'save_sensitivity_path' : save_sensitivity_path}
        
        print('Sensitivity round: ' + str(kwargs))
        rp.do_benchmarking(bi_copy, verbose, **control)
else:
    print(model + ' not implemented')

Preprocessing adult data and model for adult with random state = 123
Split data into main train-test and build forest


/home/julianhatwell/anaconda3/envs/B3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Train main model
using previous tuning parameters
Best OOB Accuracy Estimate during tuning: 0.8629
Best parameters:{'max_depth': 32.0, 'n_estimators': 600, 'oob_score': True, 'random_state': 123}


Sensitivity round: 0
Beginning benchmark for adult data.
Hyper-parameter settings: {'alpha_paths': 0.9, 'disc_path_bins': 4, 'disc_path_eqcounts': False, 'score_func': 5, 'weighting': 'kldiv', 'support_paths': 0.3, 'precis_threshold': 0.99, 'merging_bootstraps': 20, 'pruning_bootstraps': 20, 'which_trees': 'majority', 'delta': 0.09}

Prepare Unseen Data and Predictions for CHIRPS benchmark
Walking forest for 100 instances... (please wait)
Forest Walk with async = True
Forest Walk time elapsed: 4.4948 seconds

Running CHIRPS on a batch of 100 instances... (please wait)
as_chirps for batch_idx 0
start mining for batch_idx 0 with support = 0.3


/home/julianhatwell/Documents/github/explain_te/CHIRPS/structures.py:1450: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(lowers, lower_bins)[0]).round(5) # can result in nans
/home/julianhatwell/Documents/github/explain_te/CHIRPS/structures.py:1445: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(uppers, upper_bins)[0]).round(5) # can result in nans if no value falls into bin


found 29 patterns from 444 for batch_idx 0
start score sort for batch_idx 0 (29) patterns
start merge rule for batch_idx 0 (29) patterns
as_chirps for batch_idx 1
start mining for batch_idx 1 with support = 0.3


...

found 246 patterns from 343 for batch_idx 14
start score sort for batch_idx 14 (246) patterns
start merge rule for batch_idx 14 (246) patterns
[('hoursperweek', True, 42.30838), ('lcapitalgain', True, 8.01853), ('lcapitalloss', True, 7.45035), ('educationnum', True, 9.50174)]
0.9486474501108647 0.30548481576595304 0.6219261899921525 0.6310074027652658
merge complete for batch_idx 9 (498) patterns
start get explainer for batch_idx 9
[('hoursperweek', True, 31.08148), ('lcapitalgain', True, 8.55357), ('lcapitalloss', True, 7.48406), ('educationnum', True, 13.5)]
0.9566569484936832 0.14618445268367197 0.058510882087775974 0.23531499556344276
merge complete for batch_idx 11 (37) patterns
start get explainer for batch_idx 11
as_chirps for batch_idx 15
start mining for batch_idx 15 with support = 0.3


/home/julianhatwell/Documents/github/explain_te/CHIRPS/structures.py:1445: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(uppers, upper_bins)[0]).round(5) # can result in nans if no value falls into bin
/home/julianhatwell/Documents/github/explain_te/CHIRPS/structures.py:1450: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(lowers, lower_bins)[0]).round(5) # can result in nans


found 46 patterns from 600 for batch_idx 15
start score sort for batch_idx 15 (46) patterns
start merge rule for batch_idx 15 (46) patterns
as_chirps for batch_idx 16
start mining for batch_idx 16 with support = 0.3


/home/julianhatwell/Documents/github/explain_te/CHIRPS/structures.py:1450: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(lowers, lower_bins)[0]).round(5) # can result in nans
/home/julianhatwell/Documents/github/explain_te/CHIRPS/structures.py:1445: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(uppers, upper_bins)[0]).round(5) # can result in nans if no value falls into bin


found 67 patterns from 570 for batch_idx 16
start score sort for batch_idx 16 (67) patterns
start merge rule for batch_idx 16 (67) patterns
